In [ ]:
%matplotlib widget


# Annotation Capture (Versammlungsstöße)

1. YAML-Szene laden.
2. Seite aus Gretillat anzeigen.
3. Alle Referenz- und Ballpunkte nacheinander anklicken (ein Fenster reicht, Zoom bleibt erhalten).
4. Koordinaten prüfen, optional weitere Punkte hinzufügen.
5. Werte speichern und Visualisierung kontrollieren.


In [ ]:
from pathlib import Path

import numpy as np
import yaml
from PIL import Image
import matplotlib.pyplot as plt

SCENE_PATH = Path('../data/annotations/gretillat/VS-Lang-02-01.yaml')
PAGE_IMAGE = Path('../data/raw/gretillat/long_gather-185.png')

with SCENE_PATH.open() as fh:
    scene_doc = yaml.safe_load(fh)
scene_doc


In [ ]:
page_img = np.array(Image.open(PAGE_IMAGE))
plt.figure(figsize=(6, 9))
plt.imshow(page_img)
plt.axis('off')
plt.title(f"Seite {scene_doc['scene']['source']['page']}")
plt.show()


In [ ]:
try:
    import cv2
    HAS_CV2 = True
except ImportError:
    HAS_CV2 = False


def collect_points(image, prompts):
    fig, ax = plt.subplots(figsize=(6, 9))
    ax.imshow(image)
    ax.set_title(prompts[0])
    plt.show(block=False)
    print("-- Toolbar nutzen (Zoom/Pan). Mit Enter bestätigen, wenn bereit --")
    input("Enter zum Starten…")
    points = []
    for prompt in prompts:
        ax.set_title(prompt)
        plt.draw()
        click = plt.ginput(1, timeout=0)
        if not click:
            raise RuntimeError("Keine Eingabe erhalten")
        x, y = float(click[0][0]), float(click[0][1])
        points.append((x, y))
        print(f"{prompt}: Pixel=({x:.1f}, {y:.1f})")
    plt.close(fig)
    return np.array(points, dtype=float)


def compute_calibration(calib_pixels):
    pixel = np.column_stack((calib_pixels, np.ones(3)))
    table = np.array([[0.0, 0.0], [40.0, 0.0], [0.0, 80.0]])
    Mx, _, _, _ = np.linalg.lstsq(pixel, table[:, 0], rcond=None)
    My, _, _, _ = np.linalg.lstsq(pixel, table[:, 1], rcond=None)
    return np.vstack([Mx, My])


def pixel_to_table(matrix, point):
    x, y = point
    vec = np.array([x, y, 1.0])
    tx = matrix @ vec
    return float(tx[0]), float(tx[1])


def refine_center(image, guess, radius=35):
    x, y = map(int, guess)
    h, w = image.shape[:2]
    x0, x1 = max(x - radius, 0), min(x + radius, w)
    y0, y1 = max(y - radius, 0), min(y + radius, h)
    patch = image[y0:y1, x0:x1]
    if patch.size == 0:
        return guess
    gray = patch.mean(axis=2).astype(np.uint8)
    if HAS_CV2:
        blur = cv2.medianBlur(gray, 5)
        circles = cv2.HoughCircles(
            blur,
            cv2.HOUGH_GRADIENT,
            dp=1.2,
            minDist=radius * 0.8,
            param1=80,
            param2=15,
            minRadius=int(radius * 0.3),
            maxRadius=int(radius * 1.1),
        )
        if circles is not None:
            cx, cy, _ = circles[0][0]
            return x0 + float(cx), y0 + float(cy)
    thresh = gray.mean()
    mask = gray < thresh
    coords = np.column_stack(np.nonzero(mask))
    if coords.size == 0:
        return guess
    cy, cx = coords.mean(axis=0)
    return x0 + float(cx), y0 + float(cy)


def to_native(obj):
    if isinstance(obj, np.generic):
        return obj.item()
    if isinstance(obj, np.ndarray):
        return [to_native(v) for v in obj.tolist()]
    if isinstance(obj, dict):
        return {k: to_native(v) for k, v in obj.items()}
    if isinstance(obj, list):
        return [to_native(v) for v in obj]
    if isinstance(obj, tuple):
        return [to_native(v) for v in obj]
    return obj


In [ ]:
prompts = [
    "Kalibrierung 1: Ursprung (0,0)",
    "Kalibrierung 2: lange Bande (40,0)",
    "Kalibrierung 3: kurze Bande (0,80)",
    "Ball B1",
    "Ball B2",
    "Ball B3",
    "Ghost Ball"
]
points = collect_points(page_img, prompts)
calib_pixels = points[:3]
ball_pixels = points[3:]
calibration_matrix = compute_calibration(calib_pixels)

names = ['B1', 'B2', 'B3', 'Ghost']
refined_pixels = {}
table_coords = {}
for name, guess in zip(names, ball_pixels):
    refined = refine_center(page_img, guess)
    refined_pixels[name] = refined
    table_coords[name] = pixel_to_table(calibration_matrix, refined)

print("Pixelpositionen:")
for name, coords in refined_pixels.items():
    print(f"{name}: {coords}")

print("\nTischkoordinaten:")
for name, coords in table_coords.items():
    print(f"{name}: x={coords[0]:.2f}, y={coords[1]:.2f}")

extra_points = []
extra_count = int(input("Zusätzliche Punkte (Banden/Trajektorie)? Anzahl eingeben, 0 für keine: "))
if extra_count > 0:
    extra_prompts = [f"Extra Punkt {i+1}" for i in range(extra_count)]
    extra_clicks = collect_points(page_img, extra_prompts)
    for label, click in zip(extra_prompts, extra_clicks):
        refined = refine_center(page_img, click)
        coords = pixel_to_table(calibration_matrix, refined)
        extra_points.append({
            'name': label,
            'pixel': (round(refined[0], 1), round(refined[1], 1)),
            'table': (round(coords[0], 2), round(coords[1], 2)),
        })
        print(f"{label}: Tisch=({coords[0]:.2f}, {coords[1]:.2f})")
extra_points


In [ ]:
scene = scene_doc.copy()
scene_ball_data = scene['scene']['balls']
for name in ['B1', 'B2', 'B3']:
    x, y = table_coords[name]
    scene_ball_data[name]['position'] = [round(x, 2), round(y, 2)]
scene['scene']['ghost_ball']['position'] = [
    round(table_coords['Ghost'][0], 2),
    round(table_coords['Ghost'][1], 2),
]

b1 = np.array(scene_ball_data['B1']['position'])
ghost = np.array(scene['scene']['ghost_ball']['position'])
vec = ghost - b1
norm = np.linalg.norm(vec)
if norm > 0:
    scene['scene']['cue']['cue_direction'] = to_native((vec / norm).round(4).tolist())

scene['scene']['trajectory']['B1'][0]['to'] = scene['scene']['ghost_ball']['position']
scene


In [ ]:
table = scene['scene']['table']
width, height = table['size_units']
balls = scene['scene']['balls']

fig, ax = plt.subplots(figsize=(8, 4))
ax.set_xlim(0, width)
ax.set_ylim(0, height)
ax.set_aspect('equal')
ax.set_title(scene['scene']['title'])
ax.set_xlabel('Diamanten (lange Bande)')
ax.set_ylabel('Diamanten (kurze Bande)')
ax.grid(True, linestyle='--', alpha=0.3)

colors = {'white': '#ffffff', 'yellow': '#f8d93b', 'red': '#c0392b'}
ball_radius = 0.61
for name, data in balls.items():
    x, y = data['position']
    circle = plt.Circle((x, y), ball_radius, facecolor=colors.get(data['color'], '#dddddd'), edgecolor='#333333', linewidth=1.5)
    ax.add_patch(circle)
    ax.text(x, y, name, ha='center', va='center', fontsize=10)

ghost_x, ghost_y = scene['scene']['ghost_ball']['position']
ghost_circle = plt.Circle((ghost_x, ghost_y), ball_radius, facecolor='none', edgecolor='#5555ff', linestyle='--')
ax.add_patch(ghost_circle)
ax.scatter([ghost_x], [ghost_y], color='#5555ff', marker='x')
ax.text(ghost_x, ghost_y + ball_radius * 1.2, 'Ghost', ha='center', va='bottom', color='#5555ff')

b2_x, b2_y = balls['B2']['position']
ax.plot([ghost_x, b2_x], [ghost_y, b2_y], color='#5555ff', linestyle=':')

plt.show()


In [ ]:
confirm = input("Szene in YAML speichern? (y/n): ").strip().lower()
if confirm == 'y':
    with SCENE_PATH.open('w') as fh:
        yaml.safe_dump(to_native(scene), fh, allow_unicode=True, sort_keys=False)
    print("Gespeichert:", SCENE_PATH)
else:
    print("Nicht gespeichert.")
